<a href="https://colab.research.google.com/github/pkro/tensorflow_cert_training/blob/main/colab_notebooks/00_tensorflow_fundamentals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# tensor fundamentals using tensorflow

- Introduction to tensors
- getting information from tensor
- manipulating tensors
- tensors and NumPy
- Using tf.function (speed up regular python functions)
- using gpus / tpus with TensorFlow
- Exercises


Introduction to tensors


In [1]:
import tensorflow as tf

print(tf.__version__)

2022-12-25 15:19:43.593252: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-25 15:19:43.690719: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-25 15:19:43.690742: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-25 15:19:44.285813: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

2.11.0


In [2]:
print(tf.config.list_physical_devices('GPU'))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[]
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8284003673839413737
xla_global_id: -1
]


2022-12-25 15:19:46.619972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-25 15:19:46.620252: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-25 15:19:46.620329: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-25 15:19:46.620388: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-25 15:19:46.633395: W tensorflow/c

In [63]:
# Create tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [64]:
#Check number of dimentsions of a tensor (ndim = Number of DIMensions)
scalar.ndim

0

In [65]:
# create a vector
vector = tf.constant([3,4])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>

In [66]:
# check vector dimension
vector.ndim

1

In [67]:
# matrix (more than 1 dimension)
matrix = tf.constant([[10,7],
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [68]:
matrix.ndim

2

It seems the dimension of the tensors is related to the number of elements in the `shape` property of the tensor.

In [69]:
# Create another matrix and specify data tyoe
another_matrix = tf.constant([[10.,7.],
                              [8., 9.],
                              [3., 4.]], dtype=tf.float16)
another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 8.,  9.],
       [ 3.,  4.]], dtype=float16)>

In [70]:
another_matrix.ndim

2

The shape seems to have the format (rows, columns, ...);

In [71]:
# note: all of the above were tensors too as e.g. a matrix is a subtype of a tensor
tensor = tf.constant([
    [[1,2,3], [4,5,6]],
    [[7,8,9], [10,11,12]],
    [[13,14,15], [16,17,18]]
])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [72]:
tensor.ndim

3

### What we learned:

- scalar: a single number
- vector: a number with direction, e.g. wind speed and direction
- matrix: 2-dimensional array of numbers
- tensor: n-dimensional array of numbers (n = any number; a 0-dimensional tensor is a scalar, a 1-dimensional tensor is a vector)


### Creating tensors with tf.Variable and tf.constant

>A variable maintains shared, persistent state manipulated by a program.

https://www.tensorflow.org/api_docs/python/tf/Variable

When in doubt, use constant and change later if needed.

In practice, creating variables / constants is taking care of by tf.


In [73]:
v = tf.Variable(1.)
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>

In [74]:
# does not work, shape mismatch
# v.assign([1,2])
v.assign(5.) # works
v

<tf.Variable 'Variable:0' shape=() dtype=float32, numpy=5.0>

In [75]:
changeable_tensor = tf.Variable([10,7])
unchangeable_tensor = tf.constant([4,5])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([4, 5], dtype=int32)>)

In [76]:
changeable_tensor[0]

<tf.Tensor: shape=(), dtype=int32, numpy=10>

In [77]:
# changeable_tensor.assign([1,2,3]) # shape mismatch
# unchangeable_tensor.assign([1,2]) # obviousle doesn't even have an assign method
changeable_tensor.assign([1,2]) # works fine

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([1, 2], dtype=int32)>

In [78]:
# We can also assign single elements
changeable_tensor[0].assign(10)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10,  2], dtype=int32)>

### Creating random tensors with tf.Variable()

Random tensors are tensors of arbitrary size containing random numbers.

They are used to initialize the middle layer of the neural network (=representation = patterns = features = weights) and then tweak them by learning.

![random initialization](https://github.com/pkro/tensorflow_cert_training/blob/main/readme_images/random_init.png?raw=1)


In [79]:
# create 2 random (but same) tensors
random_1 = tf.random.Generator.from_seed(42); # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2)) # Outputs random values from a normal distribution.
random_1

<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
array([[-0.7565803 , -0.06854702],
       [ 0.07595026, -1.2573844 ],
       [-0.23193763, -1.8107855 ]], dtype=float32)>

>A normal distribution is a type of continuous probability distribution in which most data points cluster toward the middle of the range, while the rest taper off symmetrically toward either extreme. The middle of the range is also known as the mean of the distribution.
[source](https://www.techtarget.com/whatis/definition/normal-distribution)

Basically a bell curve.

In [80]:
random_2 = tf.random.Generator.from_seed(42)
random_2 = random_2.normal(shape=(3,2))
random_1, random_2, random_1 == random_2 # same seed, same values

(<tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=float32, numpy=
 array([[-0.7565803 , -0.06854702],
        [ 0.07595026, -1.2573844 ],
        [-0.23193763, -1.8107855 ]], dtype=float32)>,
 <tf.Tensor: shape=(3, 2), dtype=bool, numpy=
 array([[ True,  True],
        [ True,  True],
        [ True,  True]])>)

### Shuffle the order of elements in a tensor

Example use: if a NN has a list of 15000 images, the first 10.000 of Spagghetti and the last 5000 of Ramen, it might optimize too much on the Spagghetti recognition before reaching the Ramen images. It would be better to mix the input (images) so it learns both at the same time.

In [81]:
# shuffle a tensor (valuable to shuffle data so the inherent order doesn't affect learning)
not_shuffled = tf.constant([[10,7],
                            [3,5],
                            [1,9]])
not_shuffled.ndim #2
not_shuffled


<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  5],
       [ 1,  9]], dtype=int32)>

In [82]:
# shuffles only the first dimension ("rows"), content of sub-arrays stays the same!

shuffled = tf.random.shuffle(not_shuffled)
shuffled.ndim # also 2 of course
print("random, without seed:", shuffled)

# can take a seed
shuffled = tf.random.shuffle(not_shuffled, 42)
shuffled.ndim # also 2 of course
print("STILL random even with seed:", shuffled)

random, without seed: tf.Tensor(
[[ 3  5]
 [ 1  9]
 [10  7]], shape=(3, 2), dtype=int32)
STILL random even with seed: tf.Tensor(
[[10  7]
 [ 1  9]
 [ 3  5]], shape=(3, 2), dtype=int32)


>"Operations that rely on a random seed actually derive it from two seeds: the global and operation-level seeds. This sets the global seed."

[more on the rules of how these seeds are used](https://www.tensorflow.org/api_docs/python/tf/random/set_seed)

In [83]:
tf.random.set_seed(99) # global seed
shuffled = tf.random.shuffle(not_shuffled, seed=42) # operation level seed
shuffled # stays in the same (random) order

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[10,  7],
       [ 3,  5],
       [ 1,  9]], dtype=int32)>

For reproducibility of experiments, set both the global and operation level seed.

### Other ways to make tensors

### Creating tensors from NumPy arrays

Note that tensorflow has many NumPy operations (such as `ones`) already built in.

In [84]:
# creates a tensor of a given shape where all elements are 1
tf.ones(shape=(3,2), dtype=tf.int32)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[1, 1],
       [1, 1],
       [1, 1]], dtype=int32)>

In [85]:
# create a tensor of 0s
tf.zeros(shape=(3,2), dtype=tf.int32)

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[0, 0],
       [0, 0],
       [0, 0]], dtype=int32)>

In [86]:
# create a tensor filled with an arbitrary value
tf.fill([3,2], value=99) # note that shape is passed as an array instead of a tupple here

<tf.Tensor: shape=(3, 2), dtype=int32, numpy=
array([[99, 99],
       [99, 99],
       [99, 99]], dtype=int32)>

### Turn a NumPy array into tensors

Main difference between NumPy arrays and tf tensors: tensors can be run on a GPU, otherwise very similar


In [87]:
import numpy as np

numpy_a = np.arange(1, 25, dtype=np.int32)

# Capitalization "rules":
# A = tf.constant(some_matrix) # capital for matrix or tensor
# a = tf.constant(vector) # lowercase for vector
numpy_a #  numpy array

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)

In [88]:
# convert to tf tensor
tensor_a = tf.constant(numpy_a)
tensor_a

<tf.Tensor: shape=(24,), dtype=int32, numpy=
array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24], dtype=int32)>

In [89]:
# Change shape from vector to matrix from a one-dimensional array
# the number of elements in the source array must add up to the elements required
# by the shape
tensor_A = tf.constant(numpy_a, shape=(2,3,4))
tensor_A

<tf.Tensor: shape=(2, 3, 4), dtype=int32, numpy=
array([[[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]],

       [[13, 14, 15, 16],
        [17, 18, 19, 20],
        [21, 22, 23, 24]]], dtype=int32)>

### Getting information from tensors
